In [1]:
import networkx as nx
import numpy as np
from numpy.random import normal
from collections import defaultdict
import matplotlib.pyplot as plt
import os
from tqdm import tqdm 
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [10]:
class Product:
    
    def __init__(self, product_name, mean=0.5, std=0.1667, market_share=None): 
        
        '''
        Every product has a: 
            - Name
            - Basic functional quality (Q_basic) which same regardless of brands 
                - For eg, all phones should be able to call with high confidence
                - But all users have different basic qualities requirements
            - Market based quality (Q_market) which depends on market share 
            - Unknown quality is unique to every product & user
        
        Example: 
            product = Product(product_name='CCX', market_share = {'CCX':0.5, 'Canva':0.5})
        '''
        self.name = product_name
        self.market_share = market_share
                
        self.market_quality()
        self.common_qualities = {'Q_basic': normal(mean, std), 'Q_market': self.Q_market}
        
    def market_quality(self): 
        
        # Different for all products
        # This is driven by a brand marketing & Market share
        # More the market share, more is this quality expectation 
        # For example, iphones allow seamless integration with other apple devices
        
        # People will buy on this basis...
        
        shares = list(self.market_share.values())
        diff   = abs(shares[0]-shares[1])
        
        if diff==0: 
            std  = 0.01 #not confident
            mean = market_share[self.name] 
            
        else: 
            std = 0.2 #confident
            mean = market_share[self.name] + 0.2
            
        self.Q_market = normal(mean, std)    
        
    def unknown_quality(self, mean=0.5, std=0.167):
        
        # Different for all products
        # This is the qualities which may not be marketed
        # This can also be a usecase which the user finds by themselves 
        # It ranges from 0 to 1. If its 0, then there are no unknown qualities.
        return normal(mean, std) 
    

In [3]:
class Consumer:
    
    def __init__(self, consumer_id, products):

        self.products = products
        
        #personal attributes
        self.consumer_id = consumer_id 
        self.follower_tendency = normal(0.5, 0.167) 
        self.Q_personal_expectation = normal(0.3, 0.1)
        
        self.pQ_expectation  = defaultdict()
        self.pQ_hyped  = defaultdict()
        self.pQ_satisfaction = defaultdict()
         
        self.purchased  = {product.name : [False, 0, 0] for product in self.products}    

    def purchase_decision(self, Positive_i, Negative_i, N_i, N_p):
        
        '''Before purchasing the product'''
            
        for product in self.products: 
            global_qualities   = product.common_qualities
            self.pQ_expectation[product.name]  = global_qualities['Q_basic'] + global_qualities['Q_market'] + self.Q_personal_expectation
            N_prod_purchases = N_i[product.name]
            curr_Positive_i = Positive_i[product.name] + 1
            curr_Negative_i = Negative_i[product.name] + 1
            
            reviews = np.log(curr_Positive_i/curr_Negative_i)
            network_affinity = ( N_prod_purchases ** reviews ) / (N_p + 1) 
            self.pQ_hyped[product.name] = (1 - self.follower_tendency) * self.pQ_expectation[product.name] + self.follower_tendency * network_affinity
        
        
        product_estimated, product_names = [], []
        for k, v in self.pQ_hyped.items():
            product_names.append(k)
            product_estimated.append(v)
        
        purchase_product  = product_names[np.argmax(product_estimated)]
        self.purchased[purchase_product] = {True, 0, 0}
          
        for i in self.products: 
            if i.name==purchase_product:
                self.propagate_satisfaction(i)
                break
            
    
    def quality_after_use(self, product): 
        
        all_quality = {
            'common'  : product.common_qualities, 
            'personal': {'Q_unknown':  product.unknown_quality()}
        }
        
        all_quality['personal']['Q_after_use'] = normal(all_quality['personal']['Q_unknown'] + all_quality['common']['Q_basic'], 0.2) 
        
        return all_quality
        
    def quality_satisfaction(self, product, p=0.3): 
        
        '''
        Users expect the products to live upto the hype upto certain degree
        '''
        
        all_quality  = self.quality_after_use(product) 
                     
        # Has it lived upto 30% of hype 
        all_quality['personal']['lived_upto_hype'] = all_quality['personal']['Q_after_use'] - p * self.pQ_hyped[product.name]
        all_quality['personal']['product_satisfaction'] = all_quality['personal']['Q_after_use'] - self.pQ_expectation[product.name]

        return all_quality
    
    def propagate_satisfaction(self, product):

        '''After purchasing the product'''
        
        self.pQ_satisfaction[product.name] = self.quality_satisfaction(product)
        
        if self.pQ_satisfaction[product.name]['personal']['product_satisfaction'] >= 0.0 and self.pQ_satisfaction[product.name]['personal']['lived_upto_hype'] >= 0.0: 
            #convince others, positive word of mouth, negative word of mouth 
            self.purchased[product.name] = [True, 1, 0]
    
        else: 
            self.purchased[product.name] = [True, 0, 1]
        


In [4]:
class Social_Network:
    
    def __init__(self, number_of_consumers, products):
        
        self.products = products
        
        self.number_of_consumers = number_of_consumers
        self.connectedness_k =  10
        self.consumer_edge_probability = 0.2
            
        self.seed = 123
        
        self.make_social_network()
    
        
    def view_neighbours_this_product(self, product, neighbours):
        
        N_i = 0
        Pos_i = 0 
        Neg_i = 0
        for neighbour in neighbours:
            if neighbour.purchased[product.name][0]==True:
                N_i += 1
                Pos_i += neighbour.purchased[product.name][1]
                Neg_i += neighbour.purchased[product.name][2]
            
        return N_i, Pos_i, Neg_i
        
    def view_neighbourhood_all_decisions(self, neighbours): 
        
        N_p = 0
        for neighbour in neighbours:                    
            for iter_prod in self.products: 
                if neighbour.purchased[iter_prod.name]==True:
                    N_p+=1
                    
        return N_p 
    
    def make_social_network(self): 
        
        self.social_network = nx.watts_strogatz_graph(self.number_of_consumers, 
                                                      k = self.connectedness_k,
                                                      p = self.consumer_edge_probability, 
                                                      seed = self.seed)  
            
        self.neighbours = defaultdict(list)
        for u, v in self.social_network.edges:
            self.neighbours[u].append(v)
            
        self.consumer_directory = defaultdict()
        
        for consumer_id in self.social_network.nodes:
            
            consumer = Consumer(consumer_id, self.products) 
            self.consumer_directory[consumer_id] = consumer
            
        for consumer_id in self.social_network.nodes:
            
            neighbours = self.neighbours[consumer_id]
            self.neighbours[consumer_id] = [self.consumer_directory[n] for n in neighbours]
  
    def update_social_network(self): 

        N_i_all, Pos_i_all, Neg_i_all = {}, {}, {}
        for consumer_id in self.social_network.nodes:
            N_p = self.view_neighbourhood_all_decisions(self.neighbours[consumer_id])
            for product in self.products: 
                N_i, Pos_i, Neg_i = self.view_neighbours_this_product(product, self.neighbours[consumer_id])
                N_i_all[product.name] = N_i
                Pos_i_all[product.name] = Pos_i
                Neg_i_all[product.name] = Neg_i
                
            consumer = self.consumer_directory[consumer_id]             
            consumer.purchase_decision(Pos_i_all, Neg_i_all, N_i_all, N_p)
      
    def get_summary(self):
        
        lived_upto_hype = defaultdict(list)
        product_satisfaction = defaultdict(list)
        purchases = defaultdict(list)
        negative_purchases = defaultdict(list)
        positive_purchases = defaultdict(list)
        
        for consumer_id, consumer in self.consumer_directory.items():
            for product in self.products: 
                if consumer.purchased[product.name][0]:
                    lived_upto_hype[product.name].append(consumer.pQ_satisfaction[product.name]['personal']['lived_upto_hype'])
                    product_satisfaction[product.name].append(consumer.pQ_satisfaction[product.name]['personal']['product_satisfaction'])
                    purchases[product.name].append(1 if consumer.purchased[product.name][0] else 0)
                    positive_purchases[product.name].append(1 if consumer.purchased[product.name][1] else 0)
                    negative_purchases[product.name].append(1 if consumer.purchased[product.name][2] else 0)
                
         
        print()
        print_dict = {}
        for product in self.products:
            print_dict[product.name] = { 
                          "Lived upto Hype":  np.mean(lived_upto_hype[product.name]), 
                          "Product Satisfaction":  np.mean(product_satisfaction[product.name]), 
                          "# Purchases":  np.sum(purchases[product.name]), 
                          "# ++ Purchases":  np.sum(positive_purchases[product.name]), 
                          "# -- Purchases":  np.sum(negative_purchases[product.name])}
                  
        pp.pprint(print_dict)
            
        print()
        
    def save_social_network(self, timestep=0): 
        
        plt.figure(figsize =(10, 7))
  
        color = {'Canva': 'blue',
                 'CCX': 'red'}
    
        node_color = ['red' if self.consumer_directory[consumer_id].purchased['CCX'] else 'blue' for consumer_id in self.social_network.nodes]

        nx.draw_circular(self.social_network,  node_color = node_color, node_size=3)

        plt.title('Simulation Time Step t-{}'.format(str(timestep)))
        #plt.tight_layout();
                
        try:
            os.mkdir('simulations')
        except:
            pass
            #print("Folder already exists")
            
        plt.savefig('simulations/t_{}.png'.format(timestep))
        
        
        

In [5]:
def simulate(number_of_consumers=30, num_of_iterations=5, products=None): 
    
    for iteration in tqdm(range(num_of_iterations)):
        
        if iteration==0: 
            social_network = Social_Network(number_of_consumers, 
                                            products = products)
            #social_network.save_social_network(iteration) 
            social_network.get_summary()
        
        else:
            social_network.update_social_network()
            
            if iteration==num_of_iterations-1: 
                #social_network.save_social_network(iteration)
                social_network.get_summary()
            
    return social_network
        

In [6]:
#When both have same Quality distributions and Market Share
market_share = {'CCX':0.5, 'Canva':0.5}
products = [Product(product_name='CCX', market_share = market_share), Product(product_name='Canva', market_share = market_share)] #Assuming same quality distribution 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          products = products)


  0%|          | 0/200 [00:00<?, ?it/s]/Users/sshahid/causal_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/sshahid/causal_env/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 1/200 [00:03<10:50,  3.27s/it]


{   'CCX': {   '# ++ Purchases': 0.0,
               '# -- Purchases': 0.0,
               '# Purchases': 0.0,
               'Lived upto Hype': nan,
               'Product Satisfaction': nan},
    'Canva': {   '# ++ Purchases': 0.0,
                 '# -- Purchases': 0.0,
                 '# Purchases': 0.0,
                 'Lived upto Hype': nan,
                 'Product Satisfaction': nan}}



100%|██████████| 200/200 [10:23<00:00,  3.12s/it]


{   'CCX': {   '# ++ Purchases': 13406,
               '# -- Purchases': 86594,
               '# Purchases': 100000,
               'Lived upto Hype': 0.9204528172650108,
               'Product Satisfaction': -0.31014777163096957},
    'Canva': {   '# ++ Purchases': 13525,
                 '# -- Purchases': 84034,
                 '# Purchases': 97559,
                 'Lived upto Hype': 0.7128969252128652,
                 'Product Satisfaction': -0.30139824624116796}}



In [11]:
#When both have different Quality distributions and same Market Share
market_share = {'CCX':0.5, 'Canva':0.5}
products = [Product('CCX', mean = 0.5, std = 0.167, market_share = market_share), Product(product_name='Canva', mean = 0.7, std = 0.1, market_share = market_share)] 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          products = products)

  0%|          | 1/200 [00:03<13:14,  3.99s/it]


{   'CCX': {   '# ++ Purchases': 0.0,
               '# -- Purchases': 0.0,
               '# Purchases': 0.0,
               'Lived upto Hype': nan,
               'Product Satisfaction': nan},
    'Canva': {   '# ++ Purchases': 0.0,
                 '# -- Purchases': 0.0,
                 '# Purchases': 0.0,
                 'Lived upto Hype': nan,
                 'Product Satisfaction': nan}}



100%|██████████| 200/200 [09:36<00:00,  2.88s/it]


{   'CCX': {   '# ++ Purchases': 12036,
               '# -- Purchases': 76368,
               '# Purchases': 88404,
               'Lived upto Hype': 0.49463373881263417,
               'Product Satisfaction': -0.3047709691327585},
    'Canva': {   '# ++ Purchases': 14639,
                 '# -- Purchases': 85361,
                 '# Purchases': 100000,
                 'Lived upto Hype': 0.9326906091611588,
                 'Product Satisfaction': -0.2943936290553552}}



In [12]:
#When both have same Quality distributions and different Market Share
market_share = {'CCX':0.3, 'Canva':0.7}
products = [Product('CCX', market_share = market_share), Product(product_name='Canva', market_share = market_share)] #Assuming same quality distribution 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          products = products)

  0%|          | 1/200 [00:05<16:37,  5.01s/it]


{   'CCX': {   '# ++ Purchases': 0.0,
               '# -- Purchases': 0.0,
               '# Purchases': 0.0,
               'Lived upto Hype': nan,
               'Product Satisfaction': nan},
    'Canva': {   '# ++ Purchases': 0.0,
                 '# -- Purchases': 0.0,
                 '# Purchases': 0.0,
                 'Lived upto Hype': nan,
                 'Product Satisfaction': nan}}



100%|██████████| 200/200 [11:02<00:00,  3.31s/it]


{   'CCX': {   '# ++ Purchases': 27623,
               '# -- Purchases': 64130,
               '# Purchases': 91753,
               'Lived upto Hype': 0.5200864904351431,
               'Product Satisfaction': -0.14200017227793182},
    'Canva': {   '# ++ Purchases': 119,
                 '# -- Purchases': 99774,
                 '# Purchases': 99893,
                 'Lived upto Hype': 0.7760771148643731,
                 'Product Satisfaction': -0.8632026294372924}}



In [ ]:
#When both have different Quality distributions and different Market Share

market_share = {'CCX':0.3, 'Canva':0.7}
products = [Product('CCX', mean = 0.5, std = 0.167, market_share = market_share), Product(product_name='Canva', mean = 0.7, std = 0.1, market_share = market_share)] 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          products = products)
